In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from llama_index.llms.google_genai import GoogleGenAI

llm = GoogleGenAI(model="gemini-2.5-pro")

In [5]:
from google.genai import types

google_search_tool = types.Tool(
    google_search=types.GoogleSearch()
)

llm_with_search = GoogleGenAI(
    model="gemini-2.5-pro",
    generation_config=types.GenerateContentConfig(tools=[google_search_tool])
)

In [6]:
response = llm_with_search.complete("What's the stock that Motley fool was talking about today July 16?")
print(response)

On July 16, 2025, The Motley Fool discussed a variety of stocks across its platforms, highlighting opportunities in cybersecurity, banking, and dividend-paying blue-chip companies.

In the cybersecurity sector, The Motley Fool pointed to **Palo Alto Networks (PANW)**, **CrowdStrike (CRWD)**, and **SentinelOne (S)** as strong long-term investment opportunities. The analysis suggested that while all three are poised for growth, they may appeal to different investor profiles. CrowdStrike is noted for growth investors, SentinelOne for those seeking value, and Palo Alto Networks as a balanced option. The overall outlook for the cybersecurity market is positive, with an expected compound annual growth rate of 12.9% from 2025 to 2030.

Separately, a Validea report based on The Motley Fool's strategy issued upgrades for two banking stocks: **BANK7 CORP (BSVN)** and **GRUPO SUPERVIELLE SA - ADR (SUPV)**. These upgrades were based on the companies' underlying fundamentals and stock valuations.



In [7]:
from llama_index.core.workflow import Context

async def search_web(ctx: Context, query: str) -> str:
    """Useful for searching the web about a specific query or topic"""
    response = await llm_with_search.acomplete(f"""Please research given this query or topic,
    and return the result\n<query_or_topic>{query}</query_or_topic>""")
    return response

async def record_notes(ctx: Context, notes: str, notes_title: str) -> str:
    """Useful for recording notes on a given topic."""
    current_state = await ctx.store.get("state")
    if "research_notes" not in current_state:
        current_state["research_notes"] = {}
    current_state["research_notes"][notes_title] = notes
    await ctx.store.set("state", current_state)
    return "Notes recorded."

async def write_report(ctx: Context, report_content: str) -> str:
    """Useful for writing a report on a given topic."""
    current_state = await ctx.store.get("state")
    current_state["report_content"] = report_content
    await ctx.store.set("state", current_state)
    return "Report written."

async def review_report(ctx: Context, review: str) -> str:
    """Useful for reviewing a report and providing feedback."""
    current_state = await ctx.store.get("state")
    current_state["review"] = review
    await ctx.store.set("state", current_state)
    return "Report reviewed."

In [8]:
from llama_index.core.agent.workflow import (
    AgentInput,
    AgentOutput,
    ToolCall,
    ToolCallResult,
    AgentStream,
)
from llama_index.core.agent.workflow import FunctionAgent, ReActAgent

research_agent = FunctionAgent(
    name="ResearchAgent",
    description="Useful for searching the web for information on a given topic and recording notes on the topic.",
    system_prompt=(
        "You are the ResearchAgent that can search the web for information on a given topic and record notes on the topic. "
        "Once notes are recorded and you are satisfied, you should hand off control to the WriteAgent to write a report on the topic."
    ),
    llm=llm,
    tools=[search_web, record_notes],
    can_handoff_to=["WriteAgent"],
)

write_agent = FunctionAgent(
    name="WriteAgent",
    description="Useful for writing a report on a given topic.",
    system_prompt=(
        "You are the WriteAgent that can write a report on a given topic. "
        "Your report should be in a markdown format. The content should be grounded in the research notes. "
        "Once the report is written, you should get feedback at least once from the ReviewAgent."
    ),
    llm=llm,
    tools=[write_report],
    can_handoff_to=["ReviewAgent", "ResearchAgent"],
)

review_agent = FunctionAgent(
    name="ReviewAgent",
    description="Useful for reviewing a report and providing feedback.",
    system_prompt=(
        "You are the ReviewAgent that can review a report and provide feedback. "
        "Your feedback should either approve the current report or request changes for the WriteAgent to implement."
    ),
    llm=llm,
    tools=[review_report],
    can_handoff_to=["ResearchAgent","WriteAgent"],
)

In [9]:
from llama_index.core.agent.workflow import AgentWorkflow

agent_workflow = AgentWorkflow(
    agents=[research_agent, write_agent, review_agent],
    root_agent=research_agent.name,
    initial_state={
        "research_notes": {},
        "report_content": "Not written yet.",
        "review": "Review required.",
    },
)

In [12]:
from llama_index.core.agent.workflow import (
    AgentInput,
    AgentOutput,
    ToolCall,
    ToolCallResult,
    AgentStream,
)

research_topic = """Write me a report on the the future of the CRCL stock and where it is headed,
Give me a detailed insight of how in the next few days, the House of Representatives is expected to pass three measures, namely the Genius Act, the Clarity Act, and the Anti-CBDC Surveillance State Act, which, if passed into law, 
could support Circle Internet Groups (NYSE:CRCL) aggressive expansion into the digital asset sector."""

handler = agent_workflow.run(
    user_msg=research_topic
)

current_agent = None
current_tool_calls = ""
async for event in handler.stream_events():
    if (
        hasattr(event, "current_agent_name")
        and event.current_agent_name != current_agent
    ):
        current_agent = event.current_agent_name
        print(f"\n{'='*50}")
        print(f"🤖 Agent: {current_agent}")
        print(f"{'='*50}\n")
    elif isinstance(event, AgentOutput):
        if event.response.content:
            print("📤 Output:", event.response.content)
        if event.tool_calls:
            print(
                "🛠️  Planning to use tools:",
                [call.tool_name for call in event.tool_calls],
            )
    elif isinstance(event, ToolCallResult):
        print(f"🔧 Tool Result ({event.tool_name}):")
        print(f"  Arguments: {event.tool_kwargs}")
        print(f"  Output: {event.tool_output}")
    elif isinstance(event, ToolCall):
        print(f"🔨 Calling Tool: {event.tool_name}")
        print(f"  With arguments: {event.tool_kwargs}")


🤖 Agent: ResearchAgent

🛠️  Planning to use tools: ['search_web']
🔨 Calling Tool: search_web
  With arguments: {'query': 'Circle Internet Group (CRCL) stock price'}
🔧 Tool Result (search_web):
  Arguments: {'query': 'Circle Internet Group (CRCL) stock price'}
  Output: ## Circle Internet Group Now Publicly Traded on NYSE Under Ticker "CRCL"

**New York, NY –** Circle Internet Group, the company behind the popular stablecoin USD Coin (USDC), is now a publicly traded company on the New York Stock Exchange (NYSE) under the ticker symbol CRCL. This move follows the company's initial public offering (IPO) on June 5, 2025, which saw significant investor interest.

As of mid-July 2025, the stock price for Circle Internet Group (CRCL) has been trading around the $233 mark. However, the stock has experienced volatility since its debut, with a 52-week high of $298.99 and a low of $31.00.

Previously a private entity, Circle's transition to a public company marks a significant milestone for the 

In [14]:
state = await handler.ctx.store.get("state")
print("Report Content:\n", state["report_content"])
print("\n------------\nFinal Review:\n", state["review"])

Report Content:
 # Analysis of Pending Legislation and its Potential Impact on Circle Internet Group (CRCL)

## Introduction

Circle Internet Group (CRCL), the issuer of the prominent stablecoin USD Coin (USDC), recently became a publicly traded company on the New York Stock Exchange in June 2025. As a key player in the digital asset space, Circle's future is intrinsically linked to the evolving regulatory landscape. This report analyzes three significant pieces of legislation currently before the U.S. House of Representatives—the Genius Act, the Clarity Act, and the Anti-CBDC Surveillance State Act—and their potential collective impact on Circle's business and stock performance.

## The Genius Act: A Framework for Stablecoins

The "Guiding and Establishing National Innovation for U.S. Stablecoins Act," or Genius Act, is designed to establish the first comprehensive federal regulatory framework for stablecoins in the United States. For Circle, whose core product is the USDC stablecoin,

In [15]:
from IPython.display import Markdown
Markdown(state['report_content'])

# Analysis of Pending Legislation and its Potential Impact on Circle Internet Group (CRCL)

## Introduction

Circle Internet Group (CRCL), the issuer of the prominent stablecoin USD Coin (USDC), recently became a publicly traded company on the New York Stock Exchange in June 2025. As a key player in the digital asset space, Circle's future is intrinsically linked to the evolving regulatory landscape. This report analyzes three significant pieces of legislation currently before the U.S. House of Representatives—the Genius Act, the Clarity Act, and the Anti-CBDC Surveillance State Act—and their potential collective impact on Circle's business and stock performance.

## The Genius Act: A Framework for Stablecoins

The "Guiding and Establishing National Innovation for U.S. Stablecoins Act," or Genius Act, is designed to establish the first comprehensive federal regulatory framework for stablecoins in the United States. For Circle, whose core product is the USDC stablecoin, this legislation is of paramount importance.

**Key Provisions and Impact:**
*   **1:1 Asset Backing:** The act mandates that stablecoin issuers back their coins on a one-to-one basis with high-quality liquid assets, such as cash or short-term government securities. This codifies a practice that Circle already follows, lending it federal weight and legitimacy.
*   **Federal Licensing and Audits:** The requirement for issuers to obtain a federal license and undergo regular audits would create high barriers to entry, solidifying the position of established, compliant players like Circle.
*   **Regulatory Clarity:** By providing a clear set of rules, the Genius Act would significantly de-risk Circle's primary business in the eyes of institutional investors and enterprise users, potentially accelerating the adoption of USDC as a trusted digital dollar.

## The Clarity Act: Defining the Digital Asset Market

The "Digital Asset Market Clarity Act," or Clarity Act, aims to resolve the long-standing regulatory ambiguity that has hampered the digital asset industry in the U.S. It seeks to provide clear definitions for digital assets and delineate the regulatory purviews of the Securities and Exchange Commission (SEC) and the Commodity Futures Trading Commission (CFTC).

**Key Provisions and Impact:**
*   **Jurisdictional Certainty:** By assigning the SEC to oversee digital assets offered as investments and the CFTC to regulate digital commodities, the act would reduce the risk of unexpected regulatory enforcement actions that have plagued the industry.
*   **Path to Commodity Status:** The act introduces a pathway for digital assets to transition from a security to a commodity classification as they become sufficiently decentralized. This provides a clearer lifecycle for digital assets, benefiting the entire ecosystem in which Circle operates.
*   **Fostering a Stable Ecosystem:** For Circle, a clearer overall regulatory environment for digital assets means more stability for its partners and customers, from exchanges to DeFi protocols, thereby strengthening the entire value chain for USDC.

## The Anti-CBDC Surveillance State Act: Mitigating a Key Competitor

This act would prohibit the Federal Reserve from issuing a central bank digital currency (CBDC), or a "digital dollar." While not directly related to Circle's current operations, its passage would have significant strategic implications for the company's long-term future.

**Key Provisions and Impact:**
*   **Prohibition of a Federal CBDC:** The act is rooted in concerns that a government-issued digital currency could become a tool for financial surveillance and control.
*   **Elimination of a Competitor:** The primary impact on Circle is the pre-emption of a powerful, government-backed competitor. A U.S. CBDC could potentially marginalize private stablecoins like USDC. By taking a federal digital dollar off the table, this act solidifies the role of private, regulated stablecoins as the primary form of digital dollars in the U.S.

## Conclusion: A Bullish Regulatory Trifecta for CRCL

The potential passage of the Genius Act, the Clarity Act, and the Anti-CBDC Surveillance State Act represents a uniquely favorable confluence of regulatory developments for Circle Internet Group. The Genius Act would provide a clear and robust framework for its core product. The Clarity Act would bring stability and predictability to the broader industry. Finally, the Anti-CBDC Act would remove a significant potential long-term competitor.

Collectively, this legislative package would create a highly supportive regulatory environment for Circle, likely bolstering investor confidence, driving wider adoption of USDC, and providing a strong tailwind for the company's aggressive expansion into the digital asset sector. Therefore, the successful passage of these bills could be a significant bullish catalyst for CRCL's stock.